In [1]:
%matplotlib inline
import numpy as np
import sympy
from sympy import init_printing, Matrix, MatMul, integrate, symbols, expand, factor
init_printing()
from IPython.display import display, Latex, Math
import matplotlib
import matplotlib.pyplot as pl

# Design of a simple 1D Kalman filter
The following filter models the design for a Kalman filter estimating the position and speed of a falling ball, where the motion is governed by the gravity and only the position is measured.

## Setting up the environment  

In [2]:
dt = symbols('\Delta{t}')
s_0, v_0, a_0 = symbols('s_0 v_0 a_0')
a_1 = symbols('a_1')
phi_s = symbols('\Phi_s')

## State variables and control vector  


In [3]:
x0 = np.array([[s_0], [v_0]])
u1 = np.array([[a_1], [a_1]])

## State transition function  

In [4]:
F = Matrix([
    [1., dt],
    [0., 1.]
])

B = Matrix([
    [ 0.5 * dt ** 2, 0.],
    [0, dt]
])

In [5]:
F * x0 + B * u1

⎡             2                           ⎤
⎢0.5⋅\Delta{t} ⋅a₁ + \Delta{t}⋅v₀ + 1.0⋅s₀⎥
⎢                                         ⎥
⎣          \Delta{t}⋅a₁ + 1.0⋅v₀          ⎦

## Design of the Process Noise Matrix  
Discretization of the noise is modelled after: $$ \mathbf{Q} = \int_0^{\Delta t} \mathbf{F}(t)\mathbf{Q}_c \mathbf{F}^{T}(t) dt $$ where $$ \mathbf{Q}_c = \left[\begin{array}{cc}1 & 0 \\ 0 & 0\end{array}\right] \Phi_s $$
and
$$ \Phi_s $$ is the spectral density of the white noise, which will be found experimentally.

In [6]:
Q_c = Matrix([[0., 0.], [0., 1.]]) * phi_s
F_q = F * Q_c * F.transpose()
Q = sympy.integrate(F_q, (dt, 0., dt)) / phi_s
sympy.MatMul(Q, phi_s)

⎡                           3               2⎤      
⎢0.333333333333333⋅\Delta{t}   0.5⋅\Delta{t} ⎥⋅\Phiₛ
⎢                                            ⎥      
⎢                    2                       ⎥      
⎣       0.5⋅\Delta{t}          1.0⋅\Delta{t} ⎦      